# 迭代器

数据分析中重要的一个部分是经常以自动化的方式重复很多次一个类似的计算。比如，你可能有一个包含名字的表格，希望将其分割为姓和名，或者有一个包含日期的表格，可能希望将其转换为某种标准的格式。对于这类问题 Python 的答案是*迭代器 (iterator)* 语法。我们已经在 ``range`` 迭代器中见过它了：

In [1]:
for i in range(10):
    print(i, end=' ')

0 1 2 3 4 5 6 7 8 9 

在本章中我们将深入讨论迭代器。在 Python 3 中，``range`` 产生的不是一个列表，而是一个称为迭代器的东西，并且学习它的工作原理是理解许多类非常有用的 Python 功能的关键。

## 列表迭代

在对列表进行迭代的具体情况下，迭代器可能最容易理解。考虑下列代码：

In [2]:
for value in [2, 4, 6, 8, 10]:
    # 做其他一些操作
    print(value + 1, end=' ')

3 5 7 9 11 

熟悉的“``for x in y``”语法使得我们得以对列表中的每个值进行某种重复的操作。Python 代码语法与其英文描述（"*for [each] value in [the] list*"）如此接近的事实仅仅是使得 Python 成为一门非常符合直觉并且简便易学、易用语言的句法选择之一，但是在这里表面上的行为并不是**真正**发生的情况。当你写下类似“``for val in L``”这样的代码时，Python 的解释器会检查它是否包含一个*迭代器接口 (iterator interface)*。你通过内置的 ``iter`` 函数自己检查：

In [3]:
iter([2, 4, 6, 8, 10])

正是这个迭代器对象提供了 ``for`` 循环需要的功能。这个 ``iter`` 对象是一个容器，使你能访问下一个对象（只要它是合法的）。你可以通过内置的 ``next`` 函数查看：

In [4]:
I = iter([2, 4, 6, 8, 10])

In [5]:
print(next(I))

2


In [6]:
print(next(I))

4


In [7]:
print(next(I))

6


为何要这样迂回地访问对象呢？好吧，事实证明这是非常有用的。因为这样的操作允许 Python 像列表一样对待**实际上不是列表的对象**。

## ``range()``：一个不总是列表的列表

可能这种“迂回”迭代最常见的例子就是 Python 3 中的 ``range()`` 函数（在 Python 2 中这个函数命名为 ``xrange()`` ）。这个函数并不返回一个列表，而是一个特殊的 ``range()`` 对象：

In [8]:
range(10)

range(0, 10)

``range``，像一个列表一样，暴露了一个迭代器接口：

In [9]:
iter(range(10))

所以 Python 知道**假装按照**它是一个列表一样对待它：

In [10]:
for i in range(10):
    print(i, end=' ')

0 1 2 3 4 5 6 7 8 9 

这样迂回迭代的好处是，**整个列表并没有显式得到完全创建**！我们可以通过完成一个对范围进行计算的例子看到这一点。在下面这个例子中，如果我们完全实例化列表，我们的系统内存将被全部吞噬：（注意到在 Python 2 中 ``range`` 创建了一个列表，因此在 Python 2 中运行下面的代码可能导致不好的事情发生）

In [11]:
N = 10 ** 12
for i in range(N):
    if i >= 10: break
    print(i, end=', ')

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 

如果 ``range`` 真的创建了一个包含 1 万亿个值的列表，它将占用几十 TB 的机器内存：这是一个巨大的浪费，因为我们略过了除了前 10 个值之外的所有东西！

事实上，这里根本没有理由让我们对整个列表进行迭代！Python 的 ``itertools`` 库包含了一个 ``count`` 函数，这个函数起到了一个无穷 ``range`` 的作用：

In [12]:
from itertools import count

for i in count():
    if i >= 10:
        break
    print(i, end=', ')

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 

如果我们没有在这里抛出一个循环中断，它会开心地计数直到进程被手动中断或结束（比如通过 ``Ctrl-C``）。

## 实用的迭代器

这种迭代器语法几乎普遍用于 Python 的内置类型以及更多在数据科学中的特定对象，我们将在后面的部分中讨论。

这里我们将继续介绍一些 Python 语言中更有用的迭代器：

### ``enumerate``

通常，你不仅需要迭代数组中的值，还需要对索引进行跟踪。你可能会以这种方式去实现：

In [13]:
L = [2, 4, 6, 8, 10]
for i in range(len(L)):
    print(i, L[i])

0 2
1 4
2 6
3 8
4 10


尽管上述例子可行，但是 Python 提供了一个更简洁的语法，通过 ``enumerate`` 迭代器：

In [14]:
for i, val in enumerate(L):
    print(i, val)

0 2
1 4
2 6
3 8
4 10


这是一种更加 Python 的方法来对一个列表中的索引和值进行迭代。

### ``zip``

另外一些时候，你可能需要对多个列表同时进行迭代。你完全可以像之前我们看到的不 Python 的方法那样对列表索引进行迭代，但是更好的方法是使用 ``zip`` 迭代器，它将可迭代对象链在一起：

In [15]:
L = [2, 4, 6, 8, 10]
R = [3, 6, 9, 12, 15]
for lval, rval in zip(L, R):
    print(lval, rval)

2 3
4 6
6 9
8 12
10 15


任意数量的可迭代对象都可以被链在一起，其中最短的那个列表将决定整个 ``zip`` 迭代器的长度。

### ``map`` 和 ``filter``

``map`` 迭代器接受一个函数，并且将它应用到迭代器中每一个值：

In [16]:
# 寻找前 10 个平方数
square = lambda x: x ** 2
for val in map(square, range(10)):
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 

``filter`` 迭代器看上去类似，但是它只允许过滤函数为真的值保留下来：

In [17]:
# 寻找当 x % 2 为 0 且小于 10 的值
is_even = lambda x: x % 2 == 0
for val in filter(is_even, range(10)):
    print(val, end=' ')

0 2 4 6 8 

``map``、``filter`` 以及 ``reduce`` 函数（位于 Python 的 ``functools`` 模块中）是*函数式编程 (functional programming)* 风格的基本组成模块。目前在 Python 世界中，这种方式暂时还不是主流，但是仍然有着坚定的支持者（比如查阅 [pytoolz](https://toolz.readthedocs.org/en/latest/) 库）。

### 作为函数参数的迭代器

我们在第 8 章：[``*args`` 和 ``**kwargs``：灵活的参数](#*args-和-**kwargs：灵活的参数)中看见 ``*args`` 和 ``**kwargs`` 可以用来向函数传递序列和字典。事实上，``*args`` 语法不仅仅适用于序列，同样适用于任意迭代器：

In [18]:
print(*range(10))

0 1 2 3 4 5 6 7 8 9


那么，举一个例子，我们可以将之前的 ``map`` 例子用一个技巧整合进下面的函数调用中：

In [19]:
print(*map(lambda x: x ** 2, range(10)))

0 1 4 9 16 25 36 49 64 81


使用这个技巧，我们可以回答在 Python 初学者论坛上一个老生常谈的问题：为什么 Python 中没有一个 ``unzip()`` 函数恰好执行 ``zip()`` 函数相反的操作呢？如果你把自己锁在一个漆黑的地方然后思考一会儿这个问题，你可能会意识到与 ``zip()`` 相反的函数还是 ``zip()``！理解这个问题的关键在于 ``zip()`` 可以链起任意数量的迭代器或者序列。观察下列代码：

In [20]:
L1 = (1, 2, 3, 4)
L2 = ('a', 'b', 'c', 'd')

In [21]:
z = zip(L1, L2)
print(*z)

(1, 'a') (2, 'b') (3, 'c') (4, 'd')


In [22]:
z = zip(L1, L2)
new_L1, new_L2 = zip(*z)
print(new_L1, new_L2)

(1, 2, 3, 4) ('a', 'b', 'c', 'd')


好好思考一下这个例子。如果你已经理解了它是如何工作的，说明你已经深入理解了 Python 的迭代器！

## 特殊的迭代器：``itertools``

我们之前简略地介绍了无穷 ``range`` 迭代器：``itertools.count``。``itertools`` 模块包含了一系列有用的迭代器；探索这个模块来找到那些有用的迭代器是非常值得的。作为一个例子，考虑 ``itertools.permutations`` 函数，这个函数在一个序列的*全排列 (permutation)* 中进行迭代：

In [23]:
from itertools import permutations
p = permutations(range(3))
print(*p)

(0, 1, 2) (0, 2, 1) (1, 0, 2) (1, 2, 0) (2, 0, 1) (2, 1, 0)


类似地，``itertools.combinations`` 函数在一个列表中所有不同的 $N$ 组合进行迭代：

In [24]:
from itertools import combinations
c = combinations(range(4), 2)
print(*c)

(0, 1) (0, 2) (0, 3) (1, 2) (1, 3) (2, 3)


有一点儿关联的是 ``product`` 迭代器，这个迭代器对两个或更多个可迭代对象进行两两配对，最终产生一个集合：（译者注：``product`` 迭代器实际上是求*笛卡尔积 (Cartesian Product)*）

In [25]:
from itertools import product
p = product('ab', range(3))
print(*p)

('a', 0) ('a', 1) ('a', 2) ('b', 0) ('b', 1) ('b', 2)


``itertools`` 中还有更多有用的工具：你可以查阅 Python [在线文档](https://docs.python.org/3.5/library/itertools.html)找到全部的列表以及一些例子。